In [8]:
import numpy as np
from MSA_class import MultipleSequenceAlignment
from copy import deepcopy
from itertools import permutations


In [9]:
my_strings   = np.array(["AG","G"])
my_penalties = np.array([1,1,1])
my_msa       = MultipleSequenceAlignment(strings = my_strings, penalties = my_penalties) 

solutions = [my_msa.initial_bitstring,np.array([1,0,0,1,0,1],dtype=np.float64)]


print("Initial matrix:\n",my_msa.initial_MSA)

print("Valid solutions:")
for solution in solutions:
    print(my_msa.bit_state_2_matrix(solution))



Initial matrix:
 [['A' 'G']
 ['G' '_']]
Valid solutions:
[['A' 'G']
 ['G' '_']]
[['A' 'G']
 ['_' 'G']]


In [10]:
s = np.array([1 - 2 * xi for xi in my_msa.initial_bitstring]).reshape((my_msa.initial_bitstring.shape[0],1))
x = my_msa.initial_bitstring.reshape((my_msa.initial_bitstring.shape[0],1))


In [11]:
def QUBO_2_Ising(my_msa):
    Q,h,d = my_msa.QUBO_model

    J = Q / 4.

    g = np.zeros((h.shape))
    for i in range(Q.shape[0]):
        g -= (Q[:,i].reshape((Q.shape[0],1)) + (Q.T)[:,i].reshape((Q.shape[0],1)) ) / 4.

    g -=  h/2

    c = np.sum(Q) / 4. + np.sum(h) / 2. + d
    return J, g, c

Q, h, d = my_msa.QUBO_model
J, g, c = QUBO_2_Ising(my_msa)

In [12]:
#### Testing one initial state
(x.T @ Q @ x + h.T @ x +d)[0][0], (s.T @ J @ s + g.T @ s + c )[0][0]

(2.0, 2.0)

In [13]:
## Creating all permutations
state_dim = len(my_msa.initial_bitstring)
all_permutations = []
for i in range(state_dim + 1):
    current_perm = np.zeros(state_dim,dtype=int)
    for ones in range(i):
        current_perm[ones] = 1
    letter_perms = list(set(permutations(list(current_perm))))
    for perm in letter_perms:
        all_permutations.append(np.array(list(perm),dtype=int).reshape((len(perm),1)))
print("Number of permutations:",len(all_permutations))


Number of permutations: 64


In [14]:
## Checking that the two models evaluate to same cost on all perms
QUBO_costs = []
Ising_costs = []
for perm in all_permutations:
    x = perm.reshape((len(perm),1))
    s = (1-2*x)
    QUBO_costs.append((x.T @ Q @ x + h.T @ x + d)[0][0])
    Ising_costs.append((s.T @ J @ s + g.T @ s + c)[0][0])

np.array(QUBO_costs) == np.array(Ising_costs)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])